In [ ]:
import io
import itertools as it
import os
import shutil

import yaml

from snakebids import BidsComponent, BidsDataset, generate_inputs
from snakebids.tests.helpers import *


def get_component(name: str, wildcards: Mapping[str, Sequence[str]], **fixed: str):
    entities = list(wildcards.keys())
    combs = list(it.product(*wildcards.values()))
    get_zip_list(entities, combs)
    ziplist = get_zip_list(entities, combs)
    path = get_bids_path(entities, **fixed)
    return BidsComponent(name=name, path=path, zip_lists=ziplist)


full = BidsDataset.from_iterable(
    it.chain(
        (
            get_component(
                name,
                {
                    "subject": ["001", "002"],
                    "session": ["01", "02"],
                    "run": ["01"],
                },
                suffix=f"T1w{ext}",
                datatype="anat",
            )
            for name, ext in zip(("anat", "anat_meta"), (".nii.gz", ".json"))
        ),
        (
            get_component(
                name,
                {
                    "subject": ["001", "002"],
                    "session": ["01", "02"],
                    # "run": ["01"],
                },
                suffix=f"T1w{ext}",
                datatype="anat",
            )
            for name, ext in zip(("foo", "foo_meta"), (".nii.gz", ".json"))
        ),
    )
)

In [ ]:
if os.path.exists('example'):
    shutil.rmtree("example")
create_dataset("example", full)
with open("example/dataset_description.json", "w"):
    pass

In [ ]:
config = yaml.safe_load(
    io.StringIO(
        """
pybids_inputs:
  bold:
    filters:
      datatype: anat
      suffix: T1w
      extension: .nii.gz
      run:
        - 01
        - false
    wildcards:
      - subject
      - session
      - run
"""
    )
)
generate_inputs("example", config["pybids_inputs"])

In [ ]:
if os.path.exists('example'):
    shutil.rmtree("example")